In [50]:
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt
from matplotlib import patches as patches
import cv2

import os
from shutil import copy

In [51]:
train_data = pd.read_json("C:/Users/Adam/Desktop/ISA/projekty/4_DL_project/training.json")
test_data = pd.read_json("C:/Users/Adam/Desktop/ISA/projekty/4_DL_project/test.json")
print(train_data.shape, test_data.shape)

(1208, 2) (120, 2)


In [52]:
train_data.head()

,image,objects
0,{'checksum': '676bb8e86fc2dbf05dd97d51a64ac0af...,"[{'bounding_box': {'minimum': {'r': 1057, 'c':..."
1,{'checksum': '1225a18efce159eddf7b0e80e0ea642c...,"[{'bounding_box': {'minimum': {'r': 734, 'c': ..."
2,{'checksum': '3eaf840523c30fdf38897ffa01e194eb...,"[{'bounding_box': {'minimum': {'r': 724, 'c': ..."
3,{'checksum': '8a111dffacfa433029492780b9535091...,"[{'bounding_box': {'minimum': {'r': 563, 'c': ..."
4,{'checksum': 'ccef403e971460b86444cca669e68ca1...,"[{'bounding_box': {'minimum': {'r': 618, 'c': ..."


In [53]:
print(train_data['image'].sample().values, "\n",#
      train_data['objects'].sample().values)

[{'checksum': '2a740d8f0aa1ebb265b0f14fe18b0c53', 'pathname': '/images/fc8df086-39e0-43f8-b0c7-2d23a349539b.png', 'shape': {'r': 1200, 'c': 1600, 'channels': 3}}] 
 [list([{'bounding_box': {'minimum': {'r': 465, 'c': 296}, 'maximum': {'r': 578, 'c': 409}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 708, 'c': 433}, 'maximum': {'r': 831, 'c': 544}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 748, 'c': 217}, 'maximum': {'r': 863, 'c': 358}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 959, 'c': 362}, 'maximum': {'r': 1074, 'c': 494}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 1029, 'c': 75}, 'maximum': {'r': 1157, 'c': 198}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 175, 'c': 32}, 'maximum': {'r': 307, 'c': 153}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 422, 'c': 674}, 'maximum': {'r': 526, 'c': 806}}, 'category': 'red blood cell'}, {'bounding_box'

In [54]:
#splitting train set into train and validation set
#https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas

msk = np.random.rand(len(train_data)) < 0.8

train = train_data[msk]
valid = train_data[~msk]

In [55]:
train.reset_index(inplace=True)
valid.reset_index(inplace=True)

In [56]:
train.shape, valid.shape

((982, 3), (226, 3))

In [57]:
train.head()

,index,image,objects
0,0,{'checksum': '676bb8e86fc2dbf05dd97d51a64ac0af...,"[{'bounding_box': {'minimum': {'r': 1057, 'c':..."
1,2,{'checksum': '3eaf840523c30fdf38897ffa01e194eb...,"[{'bounding_box': {'minimum': {'r': 724, 'c': ..."
2,3,{'checksum': '8a111dffacfa433029492780b9535091...,"[{'bounding_box': {'minimum': {'r': 563, 'c': ..."
3,5,{'checksum': '66e3afa946245882c144dcc6af191cbe...,"[{'bounding_box': {'minimum': {'r': 230, 'c': ..."
4,7,{'checksum': '82d98ef5d434e1b2d4693baef3d6c30e...,"[{'bounding_box': {'minimum': {'r': 130, 'c': ..."


#### Creating annotation txt files for training purpose
It should be in the below form:

class_name, center_x, center_y, width, height

In [58]:
# creating dataframe with each bounding box as datapoint
train_image_list = [] # pathname
width = []            # width of the image
height = []           # height of the image
    # position of the bbox
y_max = []
y_min = []
x_max = []
x_min = []
    # label
class_label = []

for i in range(0,train.shape[0]):
    for j in range(0,len(train["objects"][i])):
        
        train_image_list.append(train["image"][i]["pathname"])
        
        width.append(train["image"][i]["shape"]["c"])
        height.append(train["image"][i]["shape"]["r"])
        
        y_max.append(train["objects"][i][j]["bounding_box"]["maximum"]["r"])
        y_min.append(train["objects"][i][j]["bounding_box"]["minimum"]["r"])
        x_max.append(train["objects"][i][j]["bounding_box"]["maximum"]["c"])
        x_min.append(train["objects"][i][j]["bounding_box"]["minimum"]["c"])
        
        class_label.append(train["objects"][i][j]["category"])
        
train_df = pd.DataFrame()

train_df["filepath"] = train_image_list
train_df["width"] = width
train_df["height"] = height
train_df["class_label"] = class_label
train_df["xmin"] = x_min
train_df["ymin"] = y_min
train_df["xmax"] = x_max
train_df["ymax"] = y_max

train_df.head()

,filepath,width,height,class_label,xmin,ymin,xmax,ymax
0,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,1440,1057,1540,1158
1,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,1303,868,1403,971
2,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,900,578,1008,689
3,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,611,304,713,408
4,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,881,198,1003,312


In [59]:
train_df["filepath"].nunique()

982

In [60]:
train_df["class_label"].value_counts()

red blood cell    62566
trophozoite        1174
difficult           359
ring                284
schizont            134
gametocyte          115
leukocyte            89
Name: class_label, dtype: int64

In [61]:
# create func to change filepath to filename
def create_filename(filepath):
    filepath = str(filepath)
    _, _, filename = filepath.split("/")
    
    return filename  

train_df.filepath = train_df.filepath.apply(create_filename)
train_df.rename(columns={"filepath": "filename"}, inplace=True)
train_df.head()

,filename,width,height,class_label,xmin,ymin,xmax,ymax
0,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,1440,1057,1540,1158
1,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,1303,868,1403,971
2,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,900,578,1008,689
3,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,611,304,713,408
4,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,881,198,1003,312


In [64]:
train_df.replace({'leukocyte': 'red blood cell', 
                  'trophozoite': 'malaria', 
                  'difficult': 'malaria', 
                  'ring': 'malaria', 
                  'schizont': 'malaria', 
                  'gametocyte': 'malaria'}, inplace=True)

In [65]:
train_df["class_label"].value_counts()

red blood cell    62655
malaria            2066
Name: class_label, dtype: int64

In [43]:
# # Create train.txt file
# print_buffer = []

# for filename in np.unique(train_df['filename'].values):
    
#     one_line = 'data/images/' + filename
#     print_buffer.append(one_line)
    
# save_file_name = os.path.join('data_images', 'train.txt')
    
# # Save the annotation to disk
# print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [75]:
# https://blog.paperspace.com/train-yolov5-custom-data/
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"red blood cell": 0,
                           "malaria": 1 
                           }

# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for bb in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[bb["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v4
        b_center_x = (bb["xmin"] + bb["xmax"]) / 2 
        b_center_y = (bb["ymin"] + bb["ymax"]) / 2
        b_width    = (bb["xmax"] - bb["xmin"])
        b_height   = (bb["ymax"] - bb["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
#     save_file_name = os.path.join("data_images_2/train", info_dict["filename"].replace("png", "txt"))
    save_file_name = os.path.join("data_images_2/valid", info_dict["filename"].replace("png", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [68]:
# Create info_dict dictionary and annotations files

for file in train_df['filename'].unique():
    
    info_dict = {}
    
    info_dict['filename'] = file
    info_dict['image_size'] = (train_df.loc[train_df['filename'] == file]['width'].unique()[0], 
                               train_df.loc[train_df['filename'] == file]['height'].unique()[0])
    
    list_bb = []
    
    for index, row in train_df.loc[train_df['filename'] == file].iterrows():
        dict_bb = {}
        dict_bb['class'] = row['class_label']
        dict_bb['xmin'] = row['xmin']
        dict_bb['ymin'] = row['ymin']
        dict_bb['xmax'] = row['xmax']
        dict_bb['ymax'] = row['ymax']
        list_bb.append(dict_bb)
        
    info_dict['bboxes'] = list_bb
    
    convert_to_yolov5(info_dict)

In [ ]:
# # Copy all training images to separate folder

# train_folder = './data_images/train'
# images_folder = './images'

# values = train_df['filename'].values
# for fname in values:
    
#     src_path = os.path.join(images_folder,fname)
#     dst_image_path = os.path.join(train_folder,fname)
    
#     # copy each image into the folder
#     copy(src_path, dst_image_path)

#### Creating annotation txt files for validation purpose
It should be in the below form:

class_name, center_x, center_y, width, height

In [69]:
# creating dataframe with each bounding box as datapoint
valid_image_list = [] # pathname
width = []            # width of the image
height = []           # height of the image
    # position of the bbox
y_max = []
y_min = []
x_max = []
x_min = []
    # label
class_label = []

for i in range(0,valid.shape[0]):
    for j in range(0,len(valid["objects"][i])):
        
        valid_image_list.append(valid["image"][i]["pathname"])
        
        width.append(valid["image"][i]["shape"]["c"])
        height.append(valid["image"][i]["shape"]["r"])
        
        y_max.append(valid["objects"][i][j]["bounding_box"]["maximum"]["r"])
        y_min.append(valid["objects"][i][j]["bounding_box"]["minimum"]["r"])
        x_max.append(valid["objects"][i][j]["bounding_box"]["maximum"]["c"])
        x_min.append(valid["objects"][i][j]["bounding_box"]["minimum"]["c"])
        
        class_label.append(valid["objects"][i][j]["category"])
        
valid_df = pd.DataFrame()

valid_df["filepath"] = valid_image_list
valid_df["width"] = width
valid_df["height"] = height
valid_df["class_label"] = class_label
valid_df["xmin"] = x_min
valid_df["ymin"] = y_min
valid_df["xmax"] = x_max
valid_df["ymax"] = y_max

valid_df.head()

,filepath,width,height,class_label,xmin,ymin,xmax,ymax
0,/images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,735,734,834,832
1,/images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,1283,939,1378,1039
2,/images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,1134,367,1235,476
3,/images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,766,307,864,400
4,/images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,1463,596,1556,701


In [70]:
valid_df["filepath"].nunique()

226

In [71]:
valid_df["class_label"].value_counts()

red blood cell    14854
trophozoite         299
difficult            82
ring                 69
schizont             45
gametocyte           29
leukocyte            14
Name: class_label, dtype: int64

In [72]:
valid_df.filepath = valid_df.filepath.apply(create_filename)
valid_df.rename(columns={"filepath": "filename"}, inplace=True)
valid_df.head()

,filename,width,height,class_label,xmin,ymin,xmax,ymax
0,10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,735,734,834,832
1,10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,1283,939,1378,1039
2,10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,1134,367,1235,476
3,10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,766,307,864,400
4,10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,1463,596,1556,701


In [73]:
valid_df.replace({'leukocyte': 'red blood cell', 
                  'trophozoite': 'malaria', 
                  'difficult': 'malaria', 
                  'ring': 'malaria', 
                  'schizont': 'malaria', 
                  'gametocyte': 'malaria'}, inplace=True)

In [74]:
valid_df["class_label"].value_counts()

red blood cell    14868
malaria             524
Name: class_label, dtype: int64

In [ ]:
# # Create test.txt file
# print_buffer = []

# for filename in np.unique(valid_df['filename'].values):
    
#     one_line = 'data/images/' + filename
#     print_buffer.append(one_line)
    
# save_file_name = os.path.join('data_images', 'test.txt')
    
# # Save the annotation to disk
# print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [76]:
# Create info_dict dictionary and annotations files

for file in valid_df['filename'].unique():
    
    info_dict = {}
    
    info_dict['filename'] = file
    info_dict['image_size'] = (valid_df.loc[valid_df['filename'] == file]['width'].unique()[0], 
                               valid_df.loc[valid_df['filename'] == file]['height'].unique()[0])
    
    list_bb = []
    
    for index, row in valid_df.loc[valid_df['filename'] == file].iterrows():
        dict_bb = {}
        dict_bb['class'] = row['class_label']
        dict_bb['xmin'] = row['xmin']
        dict_bb['ymin'] = row['ymin']
        dict_bb['xmax'] = row['xmax']
        dict_bb['ymax'] = row['ymax']
        list_bb.append(dict_bb)
        
    info_dict['bboxes'] = list_bb
    
    convert_to_yolov5(info_dict)

In [ ]:
# # Copy all validation images to separate folder

# valid_folder = './data_images/valid'
# images_folder = './images'

# values = valid_df['filename'].values
# for fname in values:
    
#     src_path = os.path.join(images_folder,fname)
#     dst_image_path = os.path.join(valid_folder,fname)
    
#     # copy each image into the folder
#     copy(src_path, dst_image_path)

#### Creating annotation txt files for validation purpose
It should be in the below form:

class_name, center_x, center_y, width, height

In [77]:
# creating dataframe with each bounding box as datapoint
test_image_list = [] # pathname
width = []            # width of the image
height = []           # height of the image
    # position of the bbox
y_max = []
y_min = []
x_max = []
x_min = []
    # label
class_label = []

for i in range(0,test_data.shape[0]):
    for j in range(0,len(test_data["objects"][i])):
        
        test_image_list.append(test_data["image"][i]["pathname"])
        
        width.append(test_data["image"][i]["shape"]["c"])
        height.append(test_data["image"][i]["shape"]["r"])
        
        y_max.append(test_data["objects"][i][j]["bounding_box"]["maximum"]["r"])
        y_min.append(test_data["objects"][i][j]["bounding_box"]["minimum"]["r"])
        x_max.append(test_data["objects"][i][j]["bounding_box"]["maximum"]["c"])
        x_min.append(test_data["objects"][i][j]["bounding_box"]["minimum"]["c"])
        
        class_label.append(test_data["objects"][i][j]["category"])
        
test_df = pd.DataFrame()

test_df["filepath"] = test_image_list
test_df["width"] = width
test_df["height"] = height
test_df["class_label"] = class_label
test_df["xmin"] = x_min
test_df["ymin"] = y_min
test_df["xmax"] = x_max
test_df["ymax"] = y_max

test_df.head()

,filepath,width,height,class_label,xmin,ymin,xmax,ymax
0,/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1744,576,1883,708
1,/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1249,863,1373,977
2,/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1573,210,1711,335
3,/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1124,1017,1253,1142
4,/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1655,572,1763,695


In [78]:
test_df["filepath"].nunique()

120

In [79]:
test_df["class_label"].value_counts()

red blood cell    5614
ring               169
trophozoite        111
gametocyte          12
schizont            11
difficult            5
Name: class_label, dtype: int64

In [80]:
test_df.filepath = test_df.filepath.apply(create_filename)
test_df.rename(columns={"filepath": "filename"}, inplace=True)
test_df.head()

,filename,width,height,class_label,xmin,ymin,xmax,ymax
0,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1744,576,1883,708
1,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1249,863,1373,977
2,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1573,210,1711,335
3,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1124,1017,1253,1142
4,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1655,572,1763,695


In [81]:
test_df.replace({'leukocyte': 'red blood cell', 
                  'trophozoite': 'malaria', 
                  'difficult': 'malaria', 
                  'ring': 'malaria', 
                  'schizont': 'malaria', 
                  'gametocyte': 'malaria'}, inplace=True)

In [82]:
test_df["class_label"].value_counts()

red blood cell    5614
malaria            308
Name: class_label, dtype: int64

In [83]:
# https://blog.paperspace.com/train-yolov5-custom-data/
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"red blood cell": 0,
                           "malaria": 1
                           }

# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for bb in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[bb["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (bb["xmin"] + bb["xmax"]) / 2 
        b_center_y = (bb["ymin"] + bb["ymax"]) / 2
        b_width    = (bb["xmax"] - bb["xmin"])
        b_height   = (bb["ymax"] - bb["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("data_images_2/test", info_dict["filename"].replace("jpg", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [84]:
# Create info_dict dictionary and annotations files

for file in test_df['filename'].unique():
    
    info_dict = {}
    
    info_dict['filename'] = file
    info_dict['image_size'] = (test_df.loc[test_df['filename'] == file]['width'].unique()[0], 
                               test_df.loc[test_df['filename'] == file]['height'].unique()[0])
    
    list_bb = []
    
    for index, row in test_df.loc[test_df['filename'] == file].iterrows():
        dict_bb = {}
        dict_bb['class'] = row['class_label']
        dict_bb['xmin'] = row['xmin']
        dict_bb['ymin'] = row['ymin']
        dict_bb['xmax'] = row['xmax']
        dict_bb['ymax'] = row['ymax']
        list_bb.append(dict_bb)
        
    info_dict['bboxes'] = list_bb
    
    convert_to_yolov5(info_dict)

In [ ]:
# # Copy all validation images to separate folder

# test_folder = './data_images/test'
# images_folder = './images'

# values = test_df['filename'].values
# for fname in values:
    
#     src_path = os.path.join(images_folder,fname)
#     dst_image_path = os.path.join(test_folder,fname)
    
#     # copy each image into the folder
#     copy(src_path, dst_image_path)

In [ ]:
# random.seed(0)

# class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

# def plot_bounding_box(image, annotation_list):
#     annotations = np.array(annotation_list)
#     w, h = image.size
    
#     plotted_image = ImageDraw.Draw(image)

#     transformed_annotations = np.copy(annotations)
#     transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
#     transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
#     transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
#     transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
#     transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
#     transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
#     for ann in transformed_annotations:
#         obj_cls, x0, y0, x1, y1 = ann
#         plotted_image.rectangle(((x0,y0), (x1,y1)))
        
#         plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))])
    
#     plt.imshow(np.array(image))
#     plt.show()

# # Get any random annotation file 
# annotation_file = random.choice(annotations)
# with open(annotation_file, "r") as file:
#     annotation_list = file.read().split("\n")[:-1]
#     annotation_list = [x.split(" ") for x in annotation_list]
#     annotation_list = [[float(y) for y in x ] for x in annotation_list]

# #Get the corresponding image file
# image_file = annotation_file.replace("annotations", "images").replace("txt", "png")
# assert os.path.exists(image_file)

# #Load the image
# image = Image.open(image_file)

# #Plot the Bounding Box
# plot_bounding_box(image, annotation_list)